# Generar Corpus en base a títulos de IL sobre Salud (IL)


In [ ]:
# Importar librerias
import os
import pandas as pd
import numpy as np
import pickle
import sweetviz as sv

# Visualización
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.lines import Line2D
import matplotlib.colors as mcolors
from matplotlib import colormaps


In [ ]:
# Configurar path
os.chdir('C://iamas_datos2024/proyectos_parlamentarios_2025/')
pd.set_option('display.max_colwidth', None)

## 1 - Recolección de datos

In [ ]:
# abrir el archivo binario proyecto filtrado
with open('proyecto_2009_2024_df_LIMPIO2.pkl', 'rb') as file:
    proyecto_2009_2024_df_LIMPIO = pickle.load(file)

#Mostrar
proyecto_2009_2024_df_LIMPIO.head()

## 2 - EDA

In [ ]:
proyecto_2009_2024_df_LIMPIO.describe(include='all').T

**Tratamiento de datos faltantes**
* Ver notebooks 2.1 y 2.2
* Duración_dias_prep, es la diferencia entre fecha de publicación y fecha maxima de movimiento úttimo. Hay valores negativos. Se los considero valores faltantes. Todos los valores faltantes se los considero -1
* Resultado. Para valores faltantes se agregaron dos categorias posible, 'NO TUVO TRATAMIENTO POSTERIOR NI DICTAMEN' o 'SIN INFORMACIÓN DE RESULTADO DE PROYECTO' (del cruzar fuentes de datos).
* Periodo, tiene valores faltantes en meses del 2009 y del 2024, que no pueden ser considerados dentro posibles períodos. La recolección se hizo por años, entre 2009 a 2024.



In [ ]:
# Controlar nulos
proyecto_2009_2024_df_LIMPIO.isnull().mean().sort_values(ascending=False)


In [ ]:
fig, axs = plt.subplots(4, 2, figsize=(30, 15))
sns.countplot(x='Tipo',  data=proyecto_2009_2024_df_LIMPIO, ax = axs[0][0])
sns.countplot(x='Proyecto_girado_a_comisiones_SALUD',  data=proyecto_2009_2024_df_LIMPIO, ax = axs[0][1])
sns.countplot(x='Proyecto_SALUD',  data=proyecto_2009_2024_df_LIMPIO, ax = axs[1][0])
sns.boxplot(data = proyecto_2009_2024_df_LIMPIO, x = 'Max_Orden', ax = axs[1][1] ) 
sns.boxplot(data = proyecto_2009_2024_df_LIMPIO, x = 'Diferencia_ID', ax = axs[2][0] ) 
sns.boxplot(data = proyecto_2009_2024_df_LIMPIO[proyecto_2009_2024_df_LIMPIO['Duración_dias_prep']>=0], x = 'Duración_dias_prep', ax = axs[2][1] ) 
sns.countplot(x='Tiene_antecedente_por_titulo_proy',  data=proyecto_2009_2024_df_LIMPIO, ax = axs[3][0])

axs[0][0].set_title('Cantidad de Iniciativas legislativas (IL) por Tipo')
axs[0][0].set_ylabel("Cantidad de IL")
axs[0][0].set_xlabel("Tipo de proyectos")
axs[0][1].set_title('Cantidad de Iniciativas legislativas (IL) según tipo de comisión asociado a SALUD')
axs[0][1].set_xlabel("COMISIONES")
axs[0][1].set_ylabel("Cantidad de IL")
axs[1][0].set_title('Cantidad de Iniciativas legislativas (IL) según tipo de comisión de cabecera asociado a SALUD')
axs[1][0].set_ylabel("Cantidad de IL")
axs[1][1].set_title(' Boxplot de Máximo orden de giro a comisión')
axs[1][1].set_xlabel("Máximo orden")
axs[1][1].set_ylabel("Cantidad de IL")
axs[2][0].set_title(' Boxplot de Diferencia_ID')
axs[2][0].set_xlabel("Diferencia de ID de IL")
axs[2][0].set_ylabel("Cantidad de IL")
axs[2][1].set_title(' Boxplot de Duración_dias (sin valores faltantes)')
axs[2][1].set_xlabel("Duración en dias")
axs[2][1].set_ylabel("Cantidad de IL")
axs[3][0].set_title('Cantidad de Iniciativas legislativas (IL) según si tiene antecedente por IL con igual titulo (sin procesar texto)')
axs[3][0].set_xlabel("Tiene antecedente de proyecto por igual título ")
axs[3][0].set_ylabel("Cantidad de IL")

fig.delaxes(axs[3][1])
plt.tight_layout(pad=2)
plt.subplots_adjust(bottom=0.2)
plt.savefig("EDA_2009_2024.png")
plt.show()






In [ ]:
# Resumen
periodo_tipo_df = pd.pivot_table(proyecto_2009_2024_df_LIMPIO, values=['Proyecto.ID'], index=['Periodo','Tipo','Proyecto_girado_a_comisiones_SALUD','Proyecto_SALUD','Tiene_antecedente_por_titulo_proy','Resultado'],
                       aggfunc={'Proyecto.ID': "count"                               
                               }).reset_index()


In [ ]:
f, axs = plt.subplots(4, 2, figsize=(40, 20), sharey = True)
sns.lineplot(x=periodo_tipo_df['Periodo'], y=periodo_tipo_df['Proyecto.ID'], estimator='sum', errorbar= None , linestyle='-', ax = axs[0][0], marker="o")
sns.lineplot(x=periodo_tipo_df['Periodo'], y=periodo_tipo_df['Proyecto.ID'], estimator='sum',  hue = 'Proyecto_girado_a_comisiones_SALUD', marker="o" ,data =periodo_tipo_df,  errorbar= None , linestyle='-',ax = axs[0][1])
sns.lineplot(x=periodo_tipo_df['Periodo'], y=periodo_tipo_df['Proyecto.ID'], estimator='sum',  hue = 'Tipo', data =periodo_tipo_df,  marker="o", errorbar= None , linestyle='-',ax = axs[1][0])
sns.lineplot(x=periodo_tipo_df['Periodo'], y=periodo_tipo_df['Proyecto.ID'], estimator='sum',  hue = 'Tipo', 
style="Proyecto_girado_a_comisiones_SALUD", data =periodo_tipo_df,  errorbar= None , linestyle='-',ax = axs[1][1], marker="o")
sns.lineplot(x=periodo_tipo_df['Periodo'], y=periodo_tipo_df['Proyecto.ID'], estimator='sum',  hue = 'Proyecto_SALUD', marker="o" ,data =periodo_tipo_df,  errorbar= None , linestyle='-',ax = axs[2][0])
sns.lineplot(x=periodo_tipo_df['Periodo'], y=periodo_tipo_df['Proyecto.ID'], estimator='sum',  hue = 'Tipo', 
style="Proyecto_SALUD", data =periodo_tipo_df,  errorbar= None , linestyle='-',ax = axs[2][1], marker="o")
sns.lineplot(x=periodo_tipo_df['Periodo'], y=periodo_tipo_df['Proyecto.ID'], estimator='sum',  hue = 'Tiene_antecedente_por_titulo_proy', marker="o" ,data =periodo_tipo_df,  errorbar= None , linestyle='-',ax = axs[3][0])
sns.lineplot(x=periodo_tipo_df['Periodo'], y=periodo_tipo_df['Proyecto.ID'], estimator='sum',  hue = 'Tiene_antecedente_por_titulo_proy', 
style="Proyecto_SALUD", data =periodo_tipo_df,  errorbar= None , linestyle='-',ax = axs[3][1], marker="o")

axs[0][0].axhline(y=periodo_tipo_df.groupby('Periodo')['Proyecto.ID'].sum().mean(), color='r', linestyle='--')
axs[0][0].set_title('Cantidad de Iniciativas legislativas (IL) por Periodo')
axs[0][0].set_xlabel("Periodo (publicación)")
axs[0][0].set_ylabel("Cantidad de IL")
axs[0][1].set_title('Cantidad de Iniciativas legislativas (IL) por Periodo y Comisión SALUD U OTRAS')
axs[0][1].set_xlabel("Periodo (publicación)")
axs[1][0].set_title('Cantidad de Iniciativas legislativas (IL) por Periodo y Tipo')
axs[1][0].set_xlabel("Periodo (publicación)")
axs[1][1].set_title('Cantidad de Iniciativas legislativas (IL) por Periodo, Tipo y Comisión SALUD U OTRAS')
axs[1][1].set_xlabel("Periodo (publicación)")
axs[2][0].set_title('Cantidad de Iniciativas legislativas (IL) por Periodo y SALUD')
axs[2][0].set_xlabel("Periodo (publicación)")
axs[2][1].set_title('Cantidad de Iniciativas legislativas (IL) por Periodo, Tipo  y SALUD')
axs[2][1].set_xlabel("Periodo (publicación)")
axs[3][0].set_title('Cantidad de Iniciativas legislativas (IL) por Periodo y tiene antecedente por titulo proy')
axs[3][0].set_xlabel("Periodo (publicación)")
axs[3][1].set_title('Cantidad de Iniciativas legislativas (IL) por Periodo, tiene antecedente por titulo proy y SALUD')
axs[3][1].set_xlabel("Periodo (publicación)")

plt.tight_layout(pad=2)
plt.subplots_adjust(bottom=0.2)

plt.show()

## 3 -Limpieza y preparación de los datos 

In [ ]:
# Librerias para texto
#import gensim
import re
#from gensim.corpora import Dictionary
import nltk
import spacy

# https://pypi.org/project/textacy/
import textacy
import textacy.preprocessing as tprep


In [ ]:
# Seleccionamos texto
texto_df = proyecto_2009_2024_df_LIMPIO[['Proyecto.ID','Título']]
texto_df.head()

Los títulos de los IL presentan entre 0 y 600 caracteres.

In [ ]:
texto_df['Título'].str.len().hist(bins='auto')

In [ ]:
texto_df['Título'].str.len().describe()

Los títulos de los IL presenta en promedio 27 palabras, min 2 palabras max 217 palabras.

In [ ]:
texto_df['Título'].str.split().map(lambda x: len(x)).describe()

In [ ]:
texto_df['Título'].str.split().map(lambda x: len(x)).hist(bins='auto')

In [ ]:
# Cantidad de palabras # tokens
proyecto_2009_2024_df_LIMPIO['Cant_palabras_aprox'] = proyecto_2009_2024_df_LIMPIO['Título'].str.split().map(lambda x: len(x))

#### Relación con variables númericas

In [ ]:
# Seleccionar solo las columnas númericas
columnas_numericas = proyecto_2009_2024_df_LIMPIO.select_dtypes(include=[np.number]).columns

In [ ]:
columnas_numericas = ['Max_Orden', 'Duración_dias_prep', 'Diferencia_ID', 'Periodo', 'Año','Cant_palabras_aprox']

In [ ]:
sns.pairplot(proyecto_2009_2024_df_LIMPIO[columnas_numericas], diag_kind="kde")


In [ ]:
# Correlación
correlaccion = proyecto_2009_2024_df_LIMPIO[columnas_numericas].corr(method='pearson')
sns.heatmap(correlaccion,xticklabels=correlaccion.columns,
yticklabels=correlaccion.columns)

#### Relación con variables categóricas

In [ ]:
proyecto_2009_2024_df_LIMPIO.columns

In [ ]:
columnas_categ = ['Proyecto_SALUD','Tipo', 'Año','Periodo','Tiene_antecedente_por_titulo_proy']

In [ ]:
# Calcular el número de filas y columnas para el subplot
n = len(columnas_categ)
nrows = 4
ncols = min(n, 2)

# Crear la figura y los subplots
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(15, 5*nrows))
fig.suptitle('Distribución de Variables Categóricas', fontsize=16)


# Aplanar el array de ejes en caso de que sea 2D
axes = axes.flatten() if n > 3 else [axes]

# Crear histogramas para cada variable numérica
for i, col in enumerate(columnas_categ):
    ax = axes[i]
    sns.violinplot(data=proyecto_2009_2024_df_LIMPIO, x=col, y="Cant_palabras_aprox", ax = ax )
    ax.set_title(f'Distribución de {col}')
    ax.set_xlabel('')
    ax.set_ylabel('Cant_palabras_aprox')
    ax.tick_params("x", rotation=45)

# Ocultar subplots vacíos si los hay
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])


plt.tight_layout(pad=2)
plt.subplots_adjust(bottom=0.2)

plt.show()


#### Correlación

In [ ]:
import scipy.stats as stats

# Prueba ANOVA y Kruskal-Wallis para comparar medias de variables numéricas según categorías#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.f_oneway.html
anova_results = {}
kruskal_results = {}
num_var = 'Cant_palabras_aprox'
for cat_var in columnas_categ:
    groups = [group[num_var].dropna() for name, group in proyecto_2009_2024_df_LIMPIO.groupby(cat_var)]
    if len(groups) > 1:
        stat, p_value_anova = stats.f_oneway(*groups)
        stat_kruskal, p_value_kruskal = stats.kruskal(*groups)
        anova_results[f'{num_var} ~ {cat_var}'] = p_value_anova
        kruskal_results[f'{num_var} ~ {cat_var}'] = p_value_kruskal

print("Resultados de la prueba ANOVA:")
for key, value in anova_results.items():
    print(f'{key}: p-value = {value}')

print("\nResultados de la prueba Kruskal-Wallis:")
for key, value in kruskal_results.items():
    print(f'{key}: p-value = {value}')

Viendo como se distribuyen los titulos por Tipo de proyecto (IL)

In [ ]:
# Explorando por Tipo de proyecto

pd.pivot_table(proyecto_2009_2024_df_LIMPIO, values=['Cant_palabras_aprox'], index=['Tipo'],
                       aggfunc={'Cant_palabras_aprox': ('mean',"min","max")}).reset_index()



In [ ]:
# Draw a nested violinplot and split the violins for easier comparison
fig, ax = plt.subplots(1,1, figsize=(15, 5))
sns.violinplot(data=proyecto_2009_2024_df_LIMPIO, x="Tipo", y="Cant_palabras_aprox", hue="Proyecto_SALUD", ax=ax)

In [ ]:
## Proyectos con titulos con dos palabras
proyecto_2009_2024_df_LIMPIO.loc[proyecto_2009_2024_df_LIMPIO['Cant_palabras_aprox'] == 2, ['Proyecto.ID','Título','Tiene_antecedente_por_titulo_proy','Tipo','Max_Orden','Periodo','Publicación.Fecha']].sort_values('Publicación.Fecha')

In [ ]:
# Analizando por cantidad promedio de palabras por, Periodo, Tipo y girado a comisiones de salud
periodo_tipo_df = pd.pivot_table(proyecto_2009_2024_df_LIMPIO, values=['Cant_palabras_aprox'], index=['Periodo','Tipo','Proyecto_SALUD'],
                       aggfunc={'Cant_palabras_aprox': "mean"}).reset_index()


In [ ]:
periodo_tipo_df.head()

In [ ]:
plt.figure(figsize=(15, 5))
g = sns.relplot(
    data=periodo_tipo_df, x="Periodo", y="Cant_palabras_aprox", col="Proyecto_SALUD",
    hue="Tipo",  kind="line", estimator = 'sum', errorbar= None, marker='o',
)

(g.set_axis_labels("Periodo", "Cantidad de palabras (media)")
  .tight_layout(w_pad=0))

plt.savefig("output.png", dpi=100, bbox_inches="tight")
plt.show()

Nos centraremos en IL por ley y exploraremos que pasa en salud. 
 * Los IL por tipo LEY presenta en promedio 21 palabras.

## NLP para IL - proyectos de ley 
#### Preprocesamiento de textos

La normalización es una tarea que tiene como objetivo poner todo el texto en igualdad de condiciones:

* ✅ Eliminar  carácteres como puntos, comas, comillas, espacios, brackets, etc.
* ✅ Convertir los números a su equivalente a palabras
* ✅ Convertir a minúsculas
* ✅ Lematizar
* ✅ Tokenizar
* ✅ Remover stopwords (opcional)
* ✅ Identificar palabras asociadas a dominio de las leyess
* ✅ Filtrar palabras asociadas a dominio

    * Tokenization: Tarea de dividir grandes cadenas de texto solo y exclusivamente en palabras.


In [ ]:
# https://github.com/RicardoMoya/NLP_with_Python/blob/master/04_Preprocesamiento_de_textos_Normalizacion.ipynb

####GENERALES####
from spacy.lang.es.stop_words import STOP_WORDS
import spacy
from collections import Counter
from itertools import islice

####DOMINIO####
#stopwords_dominio = ['numero','articulo','ley','expediente','codigo','ano','modificación','bis']  
#for word in stopwords_dominio:
#    STOP_WORDS.add(word)

# Para limpiar simbolos

import pandas as pd
import spacy

# Cargar modelo de spaCy para español
nlp = spacy.load("es_core_news_md") #  # es_core_news_md


def limpieza_basica_texto(text):
    try: 
      
       # Normaliza los caracteres Unicode del texto en formas canónicas. 
       text = tprep.normalize.unicode(text) 
       # Elimina los acentos de cualquier carácter Unicode acentuado en el texto, ya sea reemplazándolos con equivalentes ASCII o eliminándolos por completo. 
       text = tprep.remove.accents(text)
       # Por tema de leyes y números 
       text = text.replace('.','') 
       # Elimine la puntuación del texto reemplazando todas las instancias de puntuación (o un subconjunto de la misma especificada por solamente) con espacios en blanco. 
       text = tprep.remove.punctuation(text) # ojo que lo reemplaza por espacio, por eso mejor normalize.whitespace al final
       ## Elimina los espacios
       #text = text.strip() 
       # Reemplza comillas simples y dobles a solo los equivalentes básicos de ASCII. 
       text = tprep.normalize.quotation_marks(text)
       # Reemplaza espacios respetando la división entre palabras 
       text = tprep.normalize.whitespace(text) 
       # Reemplaza  {}, square [], and/or round ()
       text = tprep.remove.brackets(text)
       # Reemplaza los números por la palabra 'numero'     
       text = tprep.replace.numbers(text,"numero")
       # Pasa a minúscula
       text = text.lower()
    
    
    except:
        print("Hubo un error")
    
    return text

def get_tokens(text):
    """
    Función que dado un texto devuelve una lista con las palabras del texto no vacias
    """
    doc = nlp(text)
    return [word for word in doc if len(word.text.strip())> 0]

def remove_punctuation(words):
    """
    Función que dada una lista de palabras, elimina los signos de puntuación
    """
    doc = spacy.tokens.doc.Doc(nlp.vocab, words=words)
    return [word.text for word in doc if not word.is_punct]

def remove_short_words(words, num_chars):
    """
    Función que dada una lista de tokens y un número mínimo de caracteres que tienen que tener
    las palabras, elimina todas las palabras que tengan menos caracteres que los indicados
    """
    return " ".join([word.text for word in words if len(word.text) > num_chars])

def remove_stop_words(words):
    """
    Función que dada una lista de tokens, elimina las Stop Words
    """
    return [word for word in words if not word.is_stop]

def to_lowercase(words):
    """
    Función que dada una lista de palabras, las transforma a minúsculas
    """
    return [word.lower() for word in words]

def lemmatizer(words,  allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """
    Función que dada un texto, devuelve esa un texto con el lema de cada una de esas palabras
    """
    doc = nlp(words)
    lemma = " ".join([word.lemma_ if word.lemma_ not in ['-PRON-'] else '' for word in doc if word.pos_ in allowed_postags])
    #     " ".join([word.lemma_ if  word.lemma_ not in ['-PRON-'] else '' for word in doc])
    return lemma
    


def normalizar(text):
    """
    Dado un texto, devuelve el texto tokenizado y normalizado
    """
    # Lematizar
    words = lemmatizer(text)
    # Pasar a tokens
    words = get_tokens(text=text)
    # Eliminar stop words
    words = remove_stop_words(words)
    # Eliminar letras que pueden aparecer
    words = remove_short_words(words=words, num_chars=1)
    return words

# https://www.datacamp.com/es/tutorial/stemming-lemmatization-python
# Función para obtener n-gramas
def get_ngrams(text, n):
    doc = nlp(text)  # Procesar texto
    tokens = [token.text for token in doc if token.is_alpha and not token.is_stop]  # Filtrar palabras
    return list(zip(*[tokens[i:] for i in range(n)]))  # Generar n-gramas

# Función para eliminar los n-gramas más frecuentes
def remove_frequent_ngrams(text, ngrams_to_remove):
    doc = nlp(text)
    text_clean = text
    for ngram in ngrams_to_remove:
        text_clean = text_clean.replace(ngram, "")  # Reemplazar el n-grama con vacío
    return " ".join(text_clean.split())  # Limpiar espacios extras


In [ ]:
base_text = 'EMERGENCIA LABORAL PARA EL PERSONAL DE LA ADMINISTRACIÓN PUBLICA NACIONAL 1.343 (VER), ARTÍCULOS'
text = limpieza_basica_texto(base_text)


In [ ]:
# Probando función
print('Texto original:',base_text)
print('Limpieza primera:',text)
print("Normalizado:",normalizar(text))
print("############################# Controlando errores de normalización")
doc = nlp(text)
print('NLP spacy:',doc)
print("Token sin stop word",[word.text for word in doc if not word.is_stop])
print('-----------------')
print(doc)
print("\n ** Controlando lematizador:",[(word.lemma_ , word.pos_) for word in doc] )
allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
doc2 = nlp(" ".join([word.text for word in doc if not word.is_stop]))
for word in doc2:
    print(word.lemma_)
print("\n ** Controlando lematizador doc2:",[(word.lemma_ , word.pos_) for word in doc2] )
print("\n ** Controlando lematizador con postag:"," ".join([word.lemma_ if word.lemma_ not in ['-PRON-'] else '' for word in doc if word.pos_ in allowed_postags]))


In [ ]:
get_ngrams(normalizar(text), 2)

In [ ]:
# Seleccionanos los textos - titulos IL de ley para 2009 a 2023 -- 33052 
texto_df = proyecto_2009_2024_df_LIMPIO.loc[proyecto_2009_2024_df_LIMPIO['Tipo'] == 'LEY' , ['Proyecto.ID','Título']]#sample(frac=0.001, replace=True, random_state=1)
texto_df.info()

In [ ]:
texto_df.head(2)

In [ ]:
# Normalización
texto_df['Título normalizado'] = texto_df['Título'].copy()
texto_df['Título normalizado'] = texto_df['Título normalizado'].apply(limpieza_basica_texto) # caracteres especiales
texto_df['Título normalizado'] = texto_df['Título normalizado'].apply(normalizar)


In [ ]:
texto_df[texto_df['Título normalizado'].isna()]

#### Reducción de ruido en términos genéricos

Si ciertos n-gramas son muy frecuentes en todos los textos y no aportan información (ej. "en el año"), eliminarlos ayuda a que los embeddings reflejen conceptos más específicos del dominio.

In [ ]:
# Aplicar la limpieza al DataFrame
# Extraer n-gramas de todos los títulos

ngrams2 = []
ngrams3 = []
for titulo in texto_df["Título normalizado"]:
    ngrams2.extend(get_ngrams(titulo, 2))
    ngrams3.extend(get_ngrams(titulo, 3))

In [ ]:
# Contar los n-gramas más comunes
top_k = 50 # Número de n-gramas a eliminar
ngrams_freq2 = Counter(ngrams2).most_common(top_k)
ngrams_freq2

In [ ]:
df = pd.DataFrame.from_dict(Counter(ngrams2), orient='index', columns = ['Frecuencia']).sort_values(by = 'Frecuencia', ascending=True)
print(df.shape)
display(df.describe())
df.plot.box()

In [ ]:
# Los n-gramas son secuencias de palabras, símbolos, números o puntuación que aparecen de forma consecutiva en un texto.
fig, ax = plt.subplots(figsize=(5, 10))
df.tail(50).plot(kind='barh', ax=ax)
ax.set_title('Top 50 - 23-gramas más frecuentes')

In [ ]:
Q1 = df['Frecuencia'].quantile(0.25)
Q3 = df['Frecuencia'].quantile(0.75)
IQR = Q3 - Q1

# Definir los límites
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR
print(limite_inferior,limite_superior)

In [ ]:
# Contar los n-gramas más comunes
top_k = 50 # Número de n-gramas a eliminar
ngrams_freq = Counter(ngrams3).most_common(top_k)
ngrams_freq

In [ ]:
df = pd.DataFrame.from_dict(Counter(ngrams3), orient='index', columns = ['Frecuencia']).sort_values(by = 'Frecuencia', ascending=True)
print(df.shape)
display(df.describe())
df.plot.box()

In [ ]:
# Los n-gramas son secuencias de palabras, símbolos, números o puntuación que aparecen de forma consecutiva en un texto.
fig, ax = plt.subplots(figsize=(5, 10))
df.tail(50).plot(kind='barh', ax=ax)
ax.set_title('Top 50 - 23-gramas más frecuentes')

In [ ]:
Q1 = df['Frecuencia'].quantile(0.23)
Q3 = df['Frecuencia'].quantile(0.75)
IQR = Q3 - Q1

# Definir los límites
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR
print(limite_inferior,limite_superior)

In [ ]:
def eliminar_outliers_iqr(df, columna):
    """
    Separa los outliers y devuelve dos DataFrames:
    - df_limpio: sin outliers
    - df_outliers: con los valores atípicos detectados

    Parámetros:
    df (pd.DataFrame): DataFrame de pandas
    columna (str): Nombre de la columna a analizar

    Retorna:
    df_limpio (pd.DataFrame): DataFrame sin outliers
    df_outliers (pd.DataFrame): DataFrame solo con outliers
    """
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
   
    # Definir los límites
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    print('Método IQR')
    print('limite_inferior',limite_inferior )
    print('limite_superior',limite_superior )
    # Filtrar datos
    df_limpio = df[(df[columna] >= limite_inferior) & (df[columna] <= limite_superior)]
    df_outliers = df[(df[columna] < limite_inferior) | (df[columna] > limite_superior)]

    return df_limpio, df_outliers    

In [ ]:
#freqs_limpio_df , outliers_df = eliminar_outliers_iqr(df, 'Frecuencia')
#display("Outliers detectados:\n", outliers_df)

In [ ]:
ngrams_freq2[:7]

In [ ]:
ngrams_to_remove = {" ".join(ngram) for ngram, _ in ngrams_freq2[:7]}  # Convertir a conjunto
ngrams_to_remove

In [ ]:
texto_df['Título normalizado'] = texto_df['Título normalizado'].apply(lambda x: remove_frequent_ngrams(x, ngrams_to_remove))
texto_df['Título normalizado'] = texto_df['Título normalizado'].str.replace('numero',"")
texto_df[texto_df['Título normalizado'].isna()]

In [ ]:
texto_df['Cant_token'] = texto_df['Título'].str.split().map(lambda x: len(x))
texto_df['Cant_token_normalizado'] = texto_df['Título normalizado'].str.split().map(lambda x: len(x))


In [ ]:
# Distribución
display(texto_df[['Cant_token','Cant_token_normalizado']].describe().T)
sns.boxplot(data=texto_df[['Cant_token','Cant_token_normalizado']])

In [ ]:
# Error
Error_texto = texto_df[texto_df['Cant_token_normalizado']<1].copy()
#Error_texto['Título normalizado'] = Error_texto['Título'].apply(limpieza_basica_texto) 

In [ ]:
Error_texto.shape

Exploramos por período


In [ ]:
base_texto_df = pd.merge(texto_df,proyecto_2009_2024_df_LIMPIO[['Proyecto.ID','Proyecto_SALUD','Resultado','Periodo']], how = 'inner',left_on = 'Proyecto.ID', right_on='Proyecto.ID' )

In [ ]:
Temp = pd.pivot_table(base_texto_df, values=['Cant_token','Cant_token_normalizado'], index=['Periodo'],
                       aggfunc={'Cant_token': 'median','Cant_token_normalizado': 'median'}).reset_index()
Temp = pd.melt(Temp, id_vars=['Periodo'], value_vars=['Cant_token_normalizado', 'Cant_token'])
Temp.columns = ['Periodo','Texto','Cantidad media de palabras en título de IL']
Temp["Texto"] = Temp["Texto"].map({'Cant_token': 'Sin procesamiento', 'Cant_token_normalizado': 'Con procesamiento'})

f, ax = plt.subplots(1, 1, figsize=(15, 5), sharey = True)
sns.lineplot(x='Periodo', y='Cantidad media de palabras en título de IL', hue='Texto', data = Temp, estimator='sum', errorbar= None , linestyle='-', ax = ax, marker="o")
legend_handles, _= ax.get_legend_handles_labels()
ax.set_title('Cantidad media de palabras en título de IL por Periodo')


In [ ]:
sns.relplot(
    data=base_texto_df, x="Periodo", y="Cant_token_normalizado", col="Proyecto_SALUD",
    kind="line", estimator = 'median', errorbar= None, marker='o',
)

Exploramos por Tipo de proyecto ley, y Resultado

In [ ]:
pd.pivot_table(pd.merge(texto_df,proyecto_2009_2024_df_LIMPIO[['Proyecto.ID','Proyecto_SALUD','Resultado','Periodo']], how = 'inner',left_on = 'Proyecto.ID', right_on='Proyecto.ID' ), values=['Cant_token','Cant_token_normalizado'], index=['Resultado'],
                       aggfunc={'Cant_token': ('mean','median',"min","max"),'Cant_token_normalizado': ('mean','median',"min","max")}).reset_index()

#### Creacción del diccionario y la bolsa de palabras (BoW)
En este punto tenemos que crear:
* Corpus tokenizado: "documents_tok"
* Diccionario: "dictionary"
* Corpus: "corpus' que es la bolsa de palabras de gensim
* Detectar palabras asociado al dominio (leyes) para eliminar

In [ ]:
# https://github.com/RicardoMoya/NLP_with_Python/blob/master/21_Topic_Modeling_noticias.ipynb
# https://github.com/RicardoMoya/NLP_with_Python/blob/master/05_Bag_of_Words_BoW.ipynb

#import gensim
#import gensim.corpora as corpora
from pprint import pprint
#from gensim import corpora
from collections import defaultdict
from collections import Counter


In [ ]:
##################################PASOS DICCIONARIO########################
corpus_norm = list(texto_df['Título normalizado'])
# Tokenizamos
documents_tok = [word.split() for word in corpus_norm]

# Creamos el diccionario (vocabulario)
frequency = defaultdict(int)
for doc in documents_tok:
    for token in doc:
        frequency[token] += 1

# Exploramos todas las palabras para limpiar mejor
#counter = Counter(frequency)
frecuencia_df = pd.DataFrame.from_records(list(frequency.items()), columns = ['token','frecuencia'])
display(frecuencia_df.describe())

fig, ax = plt.subplots(figsize=(5, 10))
frecuencia_df.sort_values('frecuencia', ascending=False).head(50).plot(kind='barh', x = 'token', ax=ax)
ax.set_title('Top 50 palabras más frecuentes')


In [ ]:
frecuencia_df['frecuencia'].plot(kind='box')


In [ ]:
####GENERALES####
#from spacy.lang.es.stop_words import STOP_WORDS

####DOMINIO####
#stopwords_dominio = [
#    'numero', 'ley', 'leyes',
#    'nacional', 'modificacion',
#    'articulo', 'creacion', 'derogacion',
#    'regimen', 'declarese', 'promulgacion',
#    'expediente','codigo','nacion',
#    'ano','modificaciones','bis',
#    'incorporacion','programa','decreto','decretos',
#    'articulos','nacion'
#    ]  
#for word in stopwords_dominio:
#    STOP_WORDS.add(word)
    
#texto_df['Título normalizado'] = texto_df['Título normalizado'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords_dominio)]))


In [ ]:
import pickle

with open('texto_df_ley_0923.pkl', 'wb') as file:
    pickle.dump(texto_df,file)

In [ ]:
# Lo guardamos de nuevo
#pickle.dump(texto_df, open('./texto_df_ley_1023.pkl', 'wb'))
#texto_df = pickle.load(open('texto_df_ley_1023.pkl', 'rb'))

In [ ]:
texto_df.head(10)

In [ ]:
# Librerias para texto
import gensim
from gensim.corpora import Dictionary


In [ ]:
# Tokenizamos
corpus_norm = list(texto_df['Título normalizado'].values)
documentos_tokens = [docu.split() for docu in corpus_norm]

diccionario = Dictionary(documentos_tokens)

In [ ]:
diccionario

In [ ]:
# Diccionario
print('Diccionario Inicial: ', len(diccionario))
print(format(diccionario))

# Diccionario con más de 3 palabras (media) y con presencia de 20%
diccionario.filter_extremes(no_below=3, no_above=0.2)
diccionario.compactify()
print('Diccionario Filtrado: ', len(diccionario))
#print(dictionary.token2id)

# Creamos la Bolsa de Palabras
BoW_corpus = [diccionario.doc2bow(doc) for doc in documentos_tokens]
print('\nPrimer Documento del Corpus:\n{}'.format(BoW_corpus[0]))

In [ ]:
# Guardamos
pickle.dump(diccionario, open("diccionario_ley_0923.pkl", "wb"))
pickle.dump(BoW_corpus, open('BoW_corpus_ley_0923.pkl', 'wb'))


#### Term Frequency-Inverse Document Frequency (TF-IDF)

El TF-IDF (Frecuencia de Termino - Frecuencia Inversa de Documento) es una medida numérica que permite expresar como de relevante es una palabra para un documento en una colección de documentos (o corpus).

Construir la Bolsa de Palabras con TF-IDF en vez de con frecuencias evita dar "importancia" a texto muy largos y con mucha repetición de palabras, frente a textos cortos y con pocas repeticiones de palabras.


In [ ]:
from gensim.models import TfidfModel

# Creamos la Bolsa de Palabras con TF-ID
#BoW_corpus = [dictionary.doc2bow(doc) for doc in documents]
tfidf = TfidfModel(BoW_corpus, smartirs='ntc')
tfidf_corpus  = [tfidf[diccionario.doc2bow(doc)] for doc in documentos_tokens]

# Resultados
print('Diccionario de palabras -> palabra: id\n')
#print(dictionary_tfidf.token2id)
print('\nApariciones de las palabras en los documentos (id, tfidf):')
#tfidf_corpus

In [ ]:
# Guardamos
pickle.dump(tfidf_corpus, open('tfidf_corpus_ley_0923.pkl', 'wb'))

In [ ]:
# PARA VER palabras
filtro_texto_df = pd.merge(texto_df,proyecto_2009_2024_df_LIMPIO[['Proyecto.ID','Proyecto_girado_a_comisiones_SALUD','Proyecto_SALUD','Resultado','Max_Orden','Tiene_antecedente_por_titulo_proy','Periodo']], how = 'inner',left_on = 'Proyecto.ID', right_on='Proyecto.ID' )

In [ ]:
from tqdm import tqdm

filtro_texto_df['Tokens'] = filtro_texto_df['Título normalizado'].apply(lambda x: x.split()) #list of lists

In [ ]:
filtro_texto_df[['Título','Tokens']].head(5)

In [ ]:
# Para entender mejor un diccionario de palabras con peso por tf idf
# https://weiliu2k.github.io/CITS4012/gensim/tf-idf.html
# https://github.com/simplykeerthana/WordCloud-Generator/blob/main/word_cloud_gen.ipynb


def get_dict_to_wordcloud(documents1,dictionary):
    dic = {}
    for i, doc in enumerate(documents1):
        for idx, freq in tfidf[dictionary.doc2bow(doc)]:
            key = dictionary[idx]
            if key in dic:
                dic[key] += np.around(freq,decimals=2)
            else:
                dic[key] = np.around(freq,decimals=2)
    return dic
                
 

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import itertools
%matplotlib inline


In [ ]:
# Todo
dic_all = get_dict_to_wordcloud(documentos_tokens,diccionario)

# Salud
documentos_tokens_temp = list(filtro_texto_df[filtro_texto_df['Proyecto_SALUD']==1]['Tokens'])
dic_salud = get_dict_to_wordcloud(documentos_tokens_temp,diccionario)

# Otras
documentos_tokens_temp = list(filtro_texto_df[filtro_texto_df['Proyecto_SALUD']==0]['Tokens'])
dic_otras = get_dict_to_wordcloud(documentos_tokens_temp,diccionario)

words_comisiones = {
    ' Todos los IL': dic_all,
    'IL de Salud (girada a primera instancia a comisiones con la palabra Salud en su nombre)': dic_salud,
    'IL otro (girada a primera instancia a otras comisiones sin la palabra Salud en su nombre)':dic_otras
}

plt.figure(figsize=(25, 20))
pos = 1
for key, words in words_comisiones.items():
    plt.subplot(3, 1, pos)
    wordcloud = WordCloud(max_font_size=80, max_words=100, background_color="white").fit_words(words)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title("Palabras presentes en títulos de Iniciativas Legislativa(IL) asociadas a proyecto de Ley {}".format(key.upper()))
    pos += 1
plt.show()



¿Qué proyectos que no pasaron en primera instancia por comisión de Salud, tiene asociado la palabra reproducción?

In [ ]:
filtro_texto_df.columns

In [ ]:
reprod_df = filtro_texto_df.loc[(filtro_texto_df['Proyecto_SALUD']==0) & (filtro_texto_df['Título normalizado'].str.contains('reproduccion')== True),['Proyecto.ID', 'Título','Tokens','Proyecto_SALUD','Proyecto_girado_a_comisiones_SALUD']]

In [ ]:
reprod_df.head()

In [ ]:
reprod_df.describe(include = 'all')

In [ ]:
girocom_df = pd.read_csv('diputados/giroacomisiones2.0.csv')
print('Tamaño de dataset de Giro a comisiones:',girocom_df.shape)
display(girocom_df.head(2))


In [ ]:
reprod_df.shape

In [ ]:
temp = pd.merge(reprod_df[reprod_df['Proyecto_girado_a_comisiones_SALUD'].str.contains('GIRADO A OTRAS COMISIONES')],girocom_df[girocom_df['Orden']==1], how='inner', on= 'Proyecto.ID')
temp.shape

In [ ]:
temp.head(6)

In [ ]:
temp[temp['Proyecto_girado_a_comisiones_SALUD'].str.contains('GIRADO A OTRAS COMISIONES')]['Comisión'].value_counts()

In [ ]:
temp[temp['Comisión'].str.contains('FAMILIA, MUJER, NIÑEZ Y ADOLESCENCIA')]

In [ ]:
# Todo
dic_all = get_dict_to_wordcloud(documentos_tokens,diccionario)

# Salud
documentos_tokens_temp = list(filtro_texto_df[filtro_texto_df['Proyecto_girado_a_comisiones_SALUD']=='GIRADO A COMISIONES DE SALUD']['Tokens'])
dic_salud = get_dict_to_wordcloud(documentos_tokens_temp,diccionario)

# Otras
documentos_tokens_temp = list(filtro_texto_df[filtro_texto_df['Proyecto_girado_a_comisiones_SALUD']=='GIRADO A OTRAS COMISIONES']['Tokens'])
dic_otras = get_dict_to_wordcloud(documentos_tokens_temp,diccionario)

words_comisiones = {
    ' Todos los IL': dic_all,
    'IL girado a comisiones de Salud ': dic_salud,
    'IL girado a otras comisiones':dic_otras
}

plt.figure(figsize=(25, 20))
pos = 1
for key, words in words_comisiones.items():
    plt.subplot(3, 1, pos)
    wordcloud = WordCloud(max_font_size=80, max_words=100, background_color="white").fit_words(words)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.title("Palabras presentes en títulos de Iniciativas Legislativa(IL) asociadas a proyecto de Ley {}".format(key.upper()))
    pos += 1
plt.show()



### Común vocabulario por periodo

In [ ]:
# Guardar - OJO CON LOS REGISTROS SIN PERIODO
pickle.dump(filtro_texto_df, open('filtro_texto_df_ley_0923.pkl', 'wb'))

In [ ]:
#Agrupar titulos por periodo
periodo_salud_original_df = filtro_texto_df[filtro_texto_df['Proyecto_SALUD']==1].groupby('Periodo')['Título'].apply(lambda x: ' '.join(x)).reset_index()
periodo_salud_normalizado_df = filtro_texto_df[filtro_texto_df['Proyecto_SALUD']==1].groupby('Periodo')['Título normalizado'].apply(lambda x: ' '.join(x)).reset_index()


In [ ]:
periodo_salud_original_df.head(3)

In [ ]:
def paso_uno_pares(lista_items):
    return [(lista_items[i],lista_items[i+1]) for i in range(len(lista_items)-1)]

In [ ]:
# Ordenar períodos según el número 
periodos = sorted(periodo_salud_original_df['Periodo'].to_list())
periodo_pares = paso_uno_pares(periodos)
print(periodo_pares)

In [ ]:
def get_tokens(text):
    """
    Función que dado un texto devuelve una lista con las palabras del texto no vacias
    """
    doc = nlp(text)
    return [word.text for word in doc]

#### Calcular la intersección de vocabulario/token entre pares de períodos

In [ ]:
# Sobre titulos sin limpieza
periodo_salud_original_df['vocab'] = periodo_salud_original_df['Título'].str.lower()
periodo_salud_original_df['vocab'] = periodo_salud_original_df['vocab'].apply(get_tokens)
comun_vocab_list = []

for par in periodo_pares:
    
    s1 = set(periodo_salud_original_df.loc[(periodo_salud_original_df.Periodo==par[0])].vocab.values[0])
    s2 = set(periodo_salud_original_df.loc[(periodo_salud_original_df.Periodo==par[1])].vocab.values[0])
    comun_vocab = len(s1.intersection(s2))
    comun_vocab_list.append([par, comun_vocab])
    
comun_vocab_salud_original_df = pd.DataFrame(comun_vocab_list, columns = ['par', 'comun_vocab'])
# comun_vocabab_df_original.to_csv('../out_files/comun_vocabab_df_original.csv', index=False)
print(comun_vocab_salud_original_df.head(2))

In [ ]:
comun_vocab_salud_original_df.to_csv('./archivos_salidas/comun_vocab_salud_original.csv', index=False)


In [ ]:
periodo_salud_normalizado_df['vocab'] = periodo_salud_normalizado_df['Título normalizado'].apply(get_tokens)
comun_vocab_norm_list = []

for par in periodo_pares:
    
    s1 = set(periodo_salud_normalizado_df.loc[(periodo_salud_normalizado_df.Periodo==par[0])].vocab.values[0])
    s2 = set(periodo_salud_normalizado_df.loc[(periodo_salud_normalizado_df.Periodo==par[1])].vocab.values[0])
    comun_vocab = len(s1.intersection(s2))
    comun_vocab_norm_list.append([par, comun_vocab])
    
comun_vocab_salud_norm_df = pd.DataFrame(comun_vocab_norm_list, columns = ['par', 'comun_vocab'])
# comun_vocabab_df_original.to_csv('../out_files/comun_vocabab_df_original.csv', index=False)
print(comun_vocab_salud_norm_df.head(2))

In [ ]:
comun_vocab_salud_norm_df.to_csv('./archivos_salidas/comun_vocab_salud_norm.csv', index=False)


#### Visualizar vocabulario común

In [ ]:
periodos = {
    127: (2009, 2010),
    128: (2010, 2011),
    129: (2011, 2012),
    130: (2012, 2013),
    131: (2013, 2014),
    132: (2014, 2015),
    133: (2015, 2016),
    134: (2016, 2017),
    135: (2017, 2018),
    136: (2018, 2019),
    137: (2019, 2020),
    138: (2020, 2021),
    139: (2021, 2022),
    140: (2022, 2023)
}

comun_vocab_salud_original_df['version'] = 'Sin procesamiento'
comun_vocab_salud_original_df['Periodo inicio'] = comun_vocab_salud_original_df['par'].map(lambda x: periodos[x[0]][0])
comun_vocab_salud_original_df['Periodo fin'] = comun_vocab_salud_original_df['par'].map(lambda x: periodos[x[1]][1])
comun_vocab_salud_original_df['Periodo intermedio'] = ((comun_vocab_salud_original_df['Periodo inicio'] 
                                        + comun_vocab_salud_original_df['Periodo fin']) / 2)

comun_vocab_salud_norm_df['version'] = 'Con procesamiento'
comun_vocab_salud_norm_df['Periodo inicio'] = comun_vocab_salud_norm_df['par'].map(lambda x: periodos[x[0]][0])
comun_vocab_salud_norm_df['Periodo fin'] = comun_vocab_salud_norm_df['par'].map(lambda x: periodos[x[1]][1])
comun_vocab_salud_norm_df['Periodo intermedio'] = ((comun_vocab_salud_norm_df['Periodo inicio'] 
                                        + comun_vocab_salud_norm_df['Periodo fin']) / 2)


In [ ]:
todas_versiones_df = pd.concat([comun_vocab_salud_original_df,comun_vocab_salud_norm_df], axis=0)

In [ ]:
todas_versiones_df.shape

In [ ]:
todas_versiones_df.head()

In [ ]:
todas_versiones_df.describe()

In [ ]:
def periodos_a_fechas(periodos):

    period_dict = {
                  127:'01/03/2009-28/02/2010',
                  128:'01/03/2010-28/02/2011',
                  129:'01/03/2011-29/02/2012',
                  130:'01/03/2012-28/02/2013',
                  131:'01/03/2013-28/02/2014',
                  132:'01/03/2014-28/02/2015',
                  133:'01/03/2015-29/02/2016',
                  134:'01/03/2016-28/02/2017',
                  135:'01/03/2017-28/02/2018',
                  136:'01/03/2018-28/02/2019',
                  137:'01/03/2019-29/02/2020', 
                  138:'01/03/2020-28/02/2021', 
                  139:'01/03/2021-28/02/2022',
                  140:'01/03/2022-28/02/2023'}
    
    if isinstance(periodos, tuple):
#         return r"$\bf{" + str(periods[0]) + "\ &\ " +str(periods[1]) + "}$" + "\n" +period_dict[periods[0]]+' &\n '+period_dict[periods[1]]
        return f'{str(periodos[0])} ({period_dict[periodos[0]]}) &\n {str(periodos[1])} ({period_dict[periodos[1]]})'

    else:
        return r"$\bf{" + 'Period\ '+ str(periodos)+ "}$"+'\n'+period_dict[periodos]        


In [ ]:
from matplotlib.font_manager import FontProperties
from matplotlib.ticker import FuncFormatter
from numerize import numerize 


fig = plt.figure(figsize=(15, 6))
colors = sns.color_palette("hls", 90) #90 is available pairs

plt.rcParams.update({'font.size': 12})

versions = list(set(todas_versiones_df.version.to_list()))
                
for version in versions:
    
    Y = todas_versiones_df.loc[(todas_versiones_df['version']==version)].comun_vocab.to_list()
    X = todas_versiones_df.loc[(todas_versiones_df['version']==version)]['Periodo intermedio'].to_list()

    plt.plot(X,Y, marker="o")


xticks = comun_vocab_salud_original_df['Periodo intermedio'].unique()
xtick_labels = [ periodos_a_fechas(x) for x in sorted(comun_vocab_salud_original_df['par'].unique()) ]

plt.xlabel('Pares de Períodos Parlamentarios', fontsize=18, labelpad=140)
plt.ylabel('Número común de palabras(tokens)', fontsize=16)

ax = plt.gca()
ax.set_ylim([0,1000])
handles, labels = ax.get_legend_handles_labels()
formatter = FuncFormatter(numerize.numerize)
ax.yaxis.set_major_formatter(formatter)
plt.title('Vocabulario común entre pares de períodos', fontsize=20)
plt.legend(versions, loc='upper right',prop={'size': 14})

for i, xtick in enumerate(xticks):
    plt.annotate(xtick_labels[i], fontsize=11, annotation_clip=False,  rotation=40,  xy=(xtick, -0.5), ma='center', xytext=(xtick-2,-450))
  
    
#hide major xticks
plt.tick_params(axis='x', which='major', bottom=False, top=False, labelbottom=False)

plt.savefig('./archivos_salidas/comun_vocab.png')
plt.tight_layout(pad=2)
plt.subplots_adjust(bottom=0.2)
plt.show()
plt.cla()
plt.clf()
fig.clear()

plt.close(fig)


#### Métricas promedio para cada período parlamentario antes del preprocesamiento, después del preprocesamiento

In [ ]:
### IL de salud publicadas entres 2010-2023
filtro_texto_df[filtro_texto_df['Proyecto_SALUD']==1].shape

In [ ]:
periodo_salud_normalizado_df.shape

In [ ]:
metricas ={ 'Sin procesamiento': {
 'Cantidad promedio de palabras': round(periodo_salud_original_df['vocab'].apply(lambda x : len(x)).sum()/periodo_salud_original_df.shape[0],0),
 'Cantidad promedio de palabras únicas': round(len(set(periodo_salud_original_df.vocab.values[0]))/periodo_salud_original_df.shape[0],0) },
 'Con procesamiento': {
 'Cantidad promedio de palabras': round(periodo_salud_normalizado_df['vocab'].apply(lambda x : len(x)).sum()/periodo_salud_normalizado_df.shape[0],0),
 'Cantidad promedio de palabras únicas': round(len(set(periodo_salud_normalizado_df.vocab.values[0]))/periodo_salud_normalizado_df.shape[0],0)}}
metricas

In [ ]:
metrica_df = pd.DataFrame.from_dict(metricas, orient= 'index')
metrica_df.to_csv('./archivos_salidas/metrica_vocab_salud.csv', index=False)
metrica_df

In [ ]:
filtro_texto_df.info()

In [ ]:
proyecto_2009_2024_df_LIMPIO.columns

In [ ]:
#filtro_texto_df = filtro_texto_df = pd.merge(filtro_texto_df,proyecto_2009_2024_df_LIMPIO[['Proyecto.ID','Publicación.Fecha','Año','Duración_dias_prep']], how = 'inner',left_on = 'Proyecto.ID', right_on='Proyecto.ID' )
# Guardar
#pickle.dump(filtro_texto_df, open('filtro_texto_df_ley_1023.pkl', 'wb'))